# CP-SAT: Polynoms and Taylor expansions

In [2]:
from ortools.sat.python import cp_model
from typing import List
from math import factorial, exp


FLOAT_APPROX_PRECISION = 100

In [ ]:
from utils import add_multiplication_constraint

## Polynom with integer domain

We can compute products of variables, and in perticular power of a variable.

So we can compute polynoms. 

In [ ]:
def create_polynom(model: cp_model.CpModel, var: cp_model.IntVar, coefs: List[float], float_precision=FLOAT_APPROX_PRECISION, verbose=True):
    degree = len(coefs)

    # Approximate all coef values by mutliplying them by a big number and rounding them
    coefs = [round(float_precision * coef) for coef in coefs]

    polynom_value = 0
    polynom_value_ub = 0
    polynom_value_lb = 0
    for deg in range(degree):
        # Create the coefficient value
        if deg == 0:
            polynom_value += coefs[deg]
            polynom_value_lb += coefs[deg]
            polynom_value_ub += coefs[deg]
        elif deg == 1:
            polynom_value_lb += var.Proto().domain[0] * coefs[deg]
            polynom_value_ub += var.Proto().domain[1] * coefs[deg]

            polynom_value += coefs[deg] * var
        else:
            lb = var.Proto().domain[0] ** deg
            ub = var.Proto().domain[1] ** deg

            if (deg % 2) == 0:
                if var.Proto().domain[0] < 0:
                    lb = - lb
                if var.Proto().domain[1] < 0:
                    ub = - ub
            
            lb = coefs[deg] * lb
            ub = coefs[deg] * ub      

            polynom_value_lb += lb
            polynom_value_ub += ub 
            
            target = model.NewIntVar(lb=lb, ub=ub, name=f"{var.Name()}**{deg}")
            add_multiplication_constraint(model, target, [var]*deg)
            polynom_value += coefs[deg]*target
    
    if verbose:
        print("Polynom", polynom_value)

    polynom_var = model.NewIntVar(polynom_value_lb, polynom_value_ub, name=f"{var.Name()}_polynom")
    model.Add(polynom_var == polynom_value)
    return polynom_var

### Application : Taylor expansion

### Introduction 

Using [Taylor's theorem](https://en.wikipedia.org/wiki/Taylor%27s_theorem), we can approximate functions by polynoms using their n-th derivatives, for their values close to zero. For example, 

$$e^{x} = \sum_{k=0}^{\infty} \frac{x^k}{k!} \approx 1 + x + \frac{1}{2}x^2 + \frac{1}{6}x^3 + \frac{1}{24}x^4 + \dots$$

$$\ln(1+x) \approx x - \frac{1}{2}x^2 + \frac{1}{3}x^3 - \frac{1}{4}x^4 + \dots $$

Note that this can be used to approximate many other functions, but only close to zero. 

### Taylor approximation for integer values

Here is the polynom associated to the taylor series expansion of exp.

In [ ]:

float_precision = 10_000
polynom_degree_approx = 5
value_of_x = 1

model = cp_model.CpModel()

x = model.NewIntVar(0, 10, "x")
exp_of_x = create_polynom(model, x, coefs=[1 / factorial(k) for k in range(polynom_degree_approx)], float_precision=float_precision)

model.Add(x == value_of_x)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(f"Solver value approximation of exp({value_of_x}) is: {solver.Value(exp_of_x) / float_precision}")
print(f"Real approximation is: { sum([value_of_x**k / factorial(k) for k in range(polynom_degree_approx)]) }")
print(f"Real value is: {exp(value_of_x)}")


Polynom (((((10000 * x) + 10000) + (5000 * x**2)) + (1667 * x**3)) + (417 * x**4))
Status = OPTIMAL
Solver value approximation of exp(1) is: 2.7084
Real approximation is: 2.708333333333333
Real value is: 2.718281828459045


This can also work for negative value of exp, although we need to crank up the precision and degrees. 

In [ ]:
float_precision = 10_000_000
polynom_degree_approx = 11
value_of_x = -3

model = cp_model.CpModel()

x = model.NewIntVar(-10, 10, "x")
exp_of_x = create_polynom(model, x, coefs=[1 / factorial(k) for k in range(polynom_degree_approx)], float_precision=float_precision)

model.Add(x == value_of_x)

solver = cp_model.CpSolver()
status = solver.Solve(model)


print(f"Status = {solver.StatusName(status)}")
print(f"Solver value approximation of exp({value_of_x}) is: {solver.Value(exp_of_x) / float_precision}")
print(f"Real approximation is: { sum([value_of_x**k / factorial(k) for k in range(polynom_degree_approx)]) }")
print(f"Real value is: {exp(value_of_x)}")


Polynom (((((((((((10000000 * x) + 10000000) + (5000000 * x**2)) + (1666667 * x**3)) + (416667 * x**4)) + (83333 * x**5)) + (13889 * x**6)) + (1984 * x**7)) + (248 * x**8)) + (28 * x**9)) + (3 * x**10))
Status = OPTIMAL
Solver value approximation of exp(-3) is: 0.0539323
Real approximation is: 0.05332589285714289
Real value is: 0.049787068367863944


## Polynom on decimal domain

THe previous implementation only works to evaluate values of polynom $P(x)$ for $x \in \mathbb{Z}$, i.e. x is a signed integer. 

But we'd also like to implement polynoms for decimal values.

An intuitive way to do that would be to multiply the value of x by a big number, like we do to approximate division with decimal results. Just use `67` instead of `0.67`.

But with the current implementation, this leads to big numerical errors. For example, `67**4 + 67**2 = 20 151 121` but `0.67**4 + 0.67**2 = 0.65041121`. 

We solve this by approximating every power individually, i.e. `67**4 / 100**3 + 67**2 / 100**1 = 65`, which is much closer to what we want.

In [ ]:
def create_polynom_decimal(
    model: cp_model.CpModel,
    var: cp_model.IntVar,
    coefs: List[float],
    float_precision_var=FLOAT_APPROX_PRECISION,
    float_precision_coef=FLOAT_APPROX_PRECISION,
    verbose=True,
):
    """This polynom accepts as an input a decimal var upscaled by float_precision_var."""
    degree = len(coefs)

    # Approximate all coef values by mutliplying them by a big number and rounding them
    coefs = [round(float_precision_coef * coef) for coef in coefs]

    polynom_value = 0
    polynom_value_ub = 0
    polynom_value_lb = 0
    for deg in range(degree):
        # Create the coefficient value
        if deg == 0:
            polynom_value += coefs[deg] * float_precision_var
            polynom_value_lb += coefs[deg] * float_precision_var
            polynom_value_ub += coefs[deg] * float_precision_var
        elif deg == 1:
            polynom_value_lb += var.Proto().domain[0] * coefs[deg]
            polynom_value_ub += var.Proto().domain[1] * coefs[deg]

            polynom_value += coefs[deg] * var 
        else:
            # Bounds logic
            lb_no_coef = var.Proto().domain[0] ** deg
            ub_no_coef = var.Proto().domain[1] ** deg

            if (deg % 2) == 0:
                if var.Proto().domain[0] < 0:
                    lb_no_coef = -lb_no_coef
                if var.Proto().domain[1] < 0:
                    ub_no_coef = -ub_no_coef

            lb = coefs[deg] * lb_no_coef
            ub = coefs[deg] * ub_no_coef

            polynom_value_lb += lb
            polynom_value_ub += ub

            # Compute (x**n)
            target = model.NewIntVar(lb=lb_no_coef, ub=ub_no_coef, name=f"{var.Name()}**{deg}")
            add_multiplication_constraint(model, target, [var] * deg)
            # Then compute (a * x**n)
            target_times_coef = model.NewIntVar(
                lb=lb, ub=ub, name=f"{coefs[deg]}*{var.Name()}**{deg}"
            )
            model.Add(target_times_coef == target * coefs[deg])
            # Downscale (a * x**n) to the float_precision_var range
            target_divided_by_approx = model.NewIntVar(
                lb=lb, ub=ub, name=f"{coefs[deg]}*{var.Name()}**{deg} / ({float_precision_var**(deg-1)})"
            )
            model.AddDivisionEquality(
                target_divided_by_approx, target_times_coef, float_precision_var**(deg-1)
            )

            polynom_value += target_divided_by_approx

    if verbose:
        print("Polynom", polynom_value)

    polynom_var = model.NewIntVar(
        polynom_value_lb, polynom_value_ub, name=f"{var.Name()}_polynom"
    )
    model.Add(polynom_var == polynom_value)
    return polynom_var


### Application: Taylor series approximation for exp at decimal values

The main use of Taylor approximation is for values close to zero. Indeed, that's where the approximation is the best.

Here is the implementation with polynom evaluated at a decimal values close to zero. 

In [ ]:
float_precision_var = 100
float_precision_coefs = 1_000
polynom_degree_approx = 5
value_of_x = 0.69

model = cp_model.CpModel()

x = model.NewIntVar(-10 * float_precision_var, 10 * float_precision_var, "x")
exp_of_x = create_polynom_decimal(
    model,
    x,
    coefs=[1 / factorial(k) for k in range(polynom_degree_approx)],
    float_precision_var=float_precision_var,
    float_precision_coef=float_precision_coefs,
    verbose=True,
)

model.Add(x == round(float_precision_var * value_of_x))

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

print(
    f"Solver value approximation of exp({value_of_x}) is: {solver.Value(exp_of_x)/ float_precision_coefs / float_precision_var}"
)
print(
    f"Real approximation is: { sum([value_of_x**k / factorial(k) for k in range(polynom_degree_approx)]) }"
)
print(f"Real value is: {exp(value_of_x)}")


Polynom (((((1000 * x) + 100000) + 500*x**2 / (100)) + 167*x**3 / (10000)) + 42*x**4 / (1000000))
Status = OPTIMAL
Solver value approximation of exp(0.69) is: 1.99243
Real approximation is: 1.99224613375
Real value is: 1.9937155332430823
